<span style="color:blue">Thanks for using Drogon for your interactive Spark application. We update Drogon/SparkMagic as often as possible to make it easier, faster and more reliable for you. Have a question or feedback? Ping us on [uChat](https://uchat.uberinternal.com/uber/channels/spark).</span>

What's New
- Now you can use `%%configure` and `%%spark` magics to configure and start a Spark session (deprecating hard-to-use `%load_ext sparkmagic.magics` and `manage_spark` magics). Check out [this example](https://workbench.uberinternal.com/explore/knowledge/localfile/cwang/sparkmagic_python2_example.ipynb) for more details.
- Improved `%%configure` magic. You now can use it to make all Spark and Drogon configurations from within notebook itself. Check out our [latest documentation & examples](https://docs.google.com/document/d/1mkYtDHquh4FjqTeA0Fxii8lyV-P6qzmoABhmmRwm_00/edit#heading=h.xn14pmoorsn0) for more details.
- Bug fixes and performance updates.


In [1]:
%%configure 
{
  "pyFiles": [],
  "kind": "spark",
  "proxyUser": "radhesh",
  "sparkEnv": "SPARK_24",
  "driverMemory": "8g",
  "queue": "uber_eats_ml",
  "conf": { 
      "spark.dynamicAllocation.enabled": "true",
      "spark.dynamicAllocation.initialExecutors":100,
      "spark.dynamicAllocation.minExecutors":100,
      "spark.dynamicAllocation.maxExecutors" : 200,
      "spark.executor.memory": "12g",
      "spark.executor.memoryOverhead": "4g",
      "spark.driver.memory": "12g",
      "spark.driver.memoryOverhead" : "4g",
      "spark.hadoop.hadoop.security.authentication": "simple",
      "spark.shuffle.service.enabled" : true,
      "spark.sql.shuffle.partitions" : 500
},
  "executorCores": 2,
  "driverCores": 2,
  "executorMemory": "12g",
    "jars": ["/user/radhesh/spark-corenlp-0.4.0-spark2.4-scala2.11.jar",
            "/user/radhesh/stanford-corenlp-3.9.1-models.jar"],
   "drogonHeaders": {
    "X-DROGON-CLUSTER": "phx2/secure"
  }
}

In [2]:
spark


Starting Spark application (can take 60s or more)...
Starting heartbeat thread...done.
Waiting for Drogon session to be ready...................................................
Drogon session is ready.


Drogon Session ID,Spark Application ID,Kind,State,Spark UI,Driver log
570369339,application_1674066407532_2036689,spark,idle,Link,Link


SparkSession available as 'spark'.


Cell execution took 103 seconds.
res1: org.apache.spark.sql.SparkSession = org.apache.spark.sql.SparkSession@611052b8

In [16]:
var input_sql_query = "with data_stage0 AS ( SELECT normalized_search_term as query,item_name,COUNT(item_name) as item_freq FROM uber_eats.michelangelo_basis_eats_in_store_search_ranking_v1 where datestr >= '2022-01-01' and location_type = 'GROCERY_STORE' and merchant_type_analytics not in ('MERCHANT_TYPE_RESTAURANT','MERCHANT_TYPE_UNKNOWN') and language_code like 'en%' and length(normalized_search_term) >= 4 and length(item_name) >= 3 and country_iso3 IN ('USA', 'GBA', 'CAN') GROUP BY normalized_search_term, item_name) SELECT query, item_freq, semantic_tags from data_stage0 INNER JOIN uber_eats.semantic_tags_grocery_data ON uber_eats.semantic_tags_grocery_data.item_name = data_stage0.item_name ORDER by item_freq DESC"

input_sql_query: String = with data_stage0 AS ( SELECT normalized_search_term as query,item_name,COUNT(item_name) as item_freq FROM uber_eats.michelangelo_basis_eats_in_store_search_ranking_v1 where datestr >= '2022-01-01' and location_type = 'GROCERY_STORE' and merchant_type_analytics not in ('MERCHANT_TYPE_RESTAURANT','MERCHANT_TYPE_UNKNOWN') and language_code like 'en%' and length(normalized_search_term) >= 4 and length(item_name) >= 3 and country_iso3 IN ('USA', 'GBA', 'CAN') GROUP BY normalized_search_term, item_name) SELECT query, item_freq, semantic_tags from data_stage0 INNER JOIN uber_eats.semantic_tags_grocery_data ON uber_eats.semantic_tags_grocery_data.item_name = data_stage0.item_name ORDER by item_freq DESC

In [17]:
var data = spark.sql(input_sql_query)

data: org.apache.spark.sql.DataFrame = [query: string, item_freq: bigint ... 1 more field]

In [18]:
data.show()

+---------+---------+--------------------+
|    query|item_freq|       semantic_tags|
+---------+---------+--------------------+
|chocolate|   139654|ice cream:pre pac...|
|ice cream|   129425|ice cream:ice cre...|
|ice cream|   125503|frozen dessert:be...|
|    pizza|   123267|bakery:frozen:bre...|
|     milk|   105665|premium drink:bev...|
|ice cream|    93450|ice cream popsicl...|
|ice cream|    92837|cal dairy ice cre...|
|ice cream|    87953|ice cream:healthi...|
|ice cream|    87749|ice cream:cal dai...|
|ice cream|    87064|cal dairy:pint:ca...|
|ice cream|    85898|healthier:frozen:...|
|ice cream|    84994|ice cream popsicl...|
|     milk|    83544|snack mixer:milk:...|
|    water|    80112|         drink:water|
|ice cream|    79997|halo ice cream:ca...|
|ice cream|    79616|pint:ice cream po...|
|ice cream|    78800|yogurt smoothie k...|
|     milk|    78691|healthier drink:d...|
|ice cream|    77348|           ice cream|
|    fruit|    77071|frozen:healthier ...|
+---------+

In [19]:
data = data.withColumn("tag",explode(split(col("semantic_tags"),":")))

data: org.apache.spark.sql.DataFrame = [query: string, item_freq: bigint ... 2 more fields]

In [20]:
data.show()

+-----------+---------+--------------------+--------------------+
|      query|item_freq|       semantic_tags|                 tag|
+-----------+---------+--------------------+--------------------+
| roast beef|       17|beef pork:meat:co...|           beef pork|
| roast beef|       17|beef pork:meat:co...|                meat|
| roast beef|       17|beef pork:meat:co...|                cook|
| roast beef|       17|beef pork:meat:co...|meat seafood plan...|
| roast beef|       17|beef pork:meat:co...|        meat seafood|
|coffee mate|       17|coffee:drink:iced...|              coffee|
|coffee mate|       17|coffee:drink:iced...|               drink|
|coffee mate|       17|coffee:drink:iced...|       iced beverage|
|     hot sa|       17|consumable:cookin...|          consumable|
|     hot sa|       17|consumable:cookin...|             cooking|
|     hot sa|       17|consumable:cookin...|           condiment|
|     hot sa|       17|consumable:cookin...|noodle rice grain...|
|     hot 

In [22]:
data.drop("semantic_tags").count()

res7: Long = 271894236

In [23]:
data = data.drop("semantic_tags")

data: org.apache.spark.sql.DataFrame = [query: string, item_freq: bigint ... 1 more field]

In [24]:
data.count()

res8: Long = 271894236

In [29]:
data.show()

+------+---------+--------------------+
| query|item_freq|                 tag|
+------+---------+--------------------+
|drinks|        8|      pop cold drink|
|drinks|        8|             dog toy|
|drinks|        8|vitamin mineral s...|
|drinks|        8|               mandy|
|drinks|        8|     essential drink|
|drinks|        8|  protein supplement|
|drinks|        8|                soda|
|drinks|        8|              powder|
|drinks|        8|        milk product|
|drinks|        8|       confectionery|
|drinks|        8|             chicken|
|drinks|        8|    tea energy drink|
|drinks|        8|chill drink water...|
|drinks|        8|             alcohol|
|drinks|        8|        energy drink|
|drinks|        8|ready drink cocktail|
|drinks|        8|              health|
|drinks|        8|       internacional|
|drinks|        8|   tawny drink snack|
|drinks|        8|neighbour hood store|
+------+---------+--------------------+
only showing top 20 rows

In [37]:
var final_query_tag = data.groupBy("query","tag").agg(sum("item_freq").as("tag_freq"))

final_query_tag: org.apache.spark.sql.DataFrame = [query: string, tag: string ... 1 more field]

In [38]:
final_query_tag.show()

+--------------------+--------------------+--------+
|               query|                 tag|tag_freq|
+--------------------+--------------------+--------+
|            fig bars|               casey|      80|
|   popcorn seasoning|         snack candy|     551|
|     pantene shampoo|     harry newsagent|     195|
|              chiken|              frozen|    3539|
|              redbul|           soda shop|     824|
|  vitamin water zero|    carbonated drink|     400|
|      raspberry leaf|             alcohol|      72|
|             tequila|            beverage|  191633|
|               garba|     household paper|     254|
|               ramen|             instant|   27373|
|            box wine|               booze|    9264|
|ice cream + froze...|               water|    1578|
|   motrin children's|                baby|    1410|
|             hot dog|        chip popcorn|   12414|
|         diet sprite|           celebrate|    3407|
|      detergent pods|           household|   

In [41]:
var df = final_query_tag.orderBy("tag_freq")

df: org.apache.spark.sql.Dataset[org.apache.spark.sql.Row] = [query: string, tag: string ... 1 more field]

In [43]:
df.select(df.query,df.tag,concat_ws('-',df.tag, df.tag_freq))

<console>:26: error: value query is not a member of org.apache.spark.sql.Dataset[org.apache.spark.sql.Row]
       df.select(df.query,df.tag,concat_ws('-',df.tag, df.tag_freq))
                    ^
<console>:26: error: value tag is not a member of org.apache.spark.sql.Dataset[org.apache.spark.sql.Row]
       df.select(df.query,df.tag,concat_ws('-',df.tag, df.tag_freq))
                             ^
<console>:26: error: type mismatch;
 found   : Char('-')
 required: String
       df.select(df.query,df.tag,concat_ws('-',df.tag, df.tag_freq))
                                           ^
<console>:26: error: value tag is not a member of org.apache.spark.sql.Dataset[org.apache.spark.sql.Row]
       df.select(df.query,df.tag,concat_ws('-',df.tag, df.tag_freq))
                                                  ^
<console>:26: error: value tag_freq is not a member of org.apache.spark.sql.Dataset[org.apache.spark.sql.Row]
       df.select(df.query,df.tag,concat_ws('-',df.tag, df.tag_freq))
   

In [44]:
final_query_tag.write.mode("overwrite").insertInto("tmp.eats_query_tag_freq_grocery")

org.apache.spark.SparkException: Job aborted.
  at org.apache.spark.sql.execution.datasources.FileFormatWriter$.write(FileFormatWriter.scala:202)
  at org.apache.spark.sql.execution.datasources.InsertIntoHadoopFsRelationCommand.run(InsertIntoHadoopFsRelationCommand.scala:220)
  at org.apache.spark.sql.execution.command.DataWritingCommandExec.sideEffectResult$lzycompute(commands.scala:105)
  at org.apache.spark.sql.execution.command.DataWritingCommandExec.sideEffectResult(commands.scala:103)
  at org.apache.spark.sql.execution.command.DataWritingCommandExec.doExecute(commands.scala:131)
  at org.apache.spark.sql.execution.SparkPlan$$anonfun$execute$1.apply(SparkPlan.scala:131)
  at org.apache.spark.sql.execution.SparkPlan$$anonfun$execute$1.apply(SparkPlan.scala:127)
  at org.apache.spark.sql.execution.SparkPlan$$anonfun$executeQuery$1.apply(SparkPlan.scala:155)
  at org.apache.spark.rdd.RDDOperationScope$.withScope(RDDOperationScope.scala:151)
  at org.apache.spark.sql.execution.SparkP